In [12]:
import calliope
import plotly.express as px

Run the model for a week

In [7]:
model = calliope.Model("build/model/model.yaml", time_subset=['2016-01-01', '2016-01-31'])
model.build()
model.solve()

Postprocessing: All values < 1e-10 set to 0 in cost_var


In [8]:
model.results.techs

<xarray.DataArray 'techs' (techs: 12)> Size: 96B
array(['battery', 'demand_elec', 'hydro_reservoir', 'hydro_run_of_river',
       'hydrogen', 'nuclear', 'open_field_pv', 'pumped_hydro',
       'roof_mounted_pv', 'wind_offshore', 'wind_onshore_competing',
       'wind_onshore_monopoly'], dtype=object)
Coordinates:
  * techs    (techs) object 96B 'battery' ... 'wind_onshore_monopoly'

In [11]:
model._model_data

<xarray.Dataset> Size: 12MB
Dimensions:                      (nodes: 34, techs: 12, carriers: 1,
                                  timesteps: 336, costs: 1)
Coordinates:
  * techs                        (techs) object 96B 'battery' ... 'wind_onsho...
  * carriers                     (carriers) <U11 44B 'electricity'
  * nodes                        (nodes) object 272B 'ALB' 'AUT' ... 'SVN' 'SWE'
  * timesteps                    (timesteps) datetime64[ns] 3kB 2016-01-01 .....
  * costs                        (costs) object 8B 'monetary'
Data variables: (12/54)
    flow_cap                     (nodes, techs, carriers) float64 3kB 0.00274...
    flow_out                     (nodes, techs, carriers, timesteps) float64 1MB ...
    flow_in                      (nodes, techs, carriers, timesteps) float64 1MB ...
    area_use                     (nodes, techs) float64 3kB nan nan ... 0.0 nan
    source_use                   (nodes, techs, timesteps) float64 1MB nan .....
    source_cap                   (nodes, techs) float64 3kB nan nan ... 0.07829
    ...                           ...
    source_use_max               (techs, timesteps, nodes) float64 1MB nan .....
    definition_matrix            (nodes, techs, carriers) bool 408B True ... ...
    color                        (techs) <U7 336B '#19122b' ... '#17344c'
    distance                     (techs) float64 96B nan nan nan ... nan nan nan
    timestep_resolution          (timesteps) float64 3kB 1.0 1.0 1.0 ... 1.0 1.0
    timestep_weights             (timesteps) float64 3kB 1.0 1.0 1.0 ... 1.0 1.0
Attributes: (12/16)
    termination_condition:         optimal
    calliope_version_defined:      0.7.0
    calliope_version_initialised:  0.7.0.dev4
    applied_overrides:             
    scenario:                      None
    defaults:                      {'bigM': 1000000000.0, 'objective_cost_wei...
    ...                            ...
    name:                          Example model
    timestamp_model_creation:      1726484997.466857
    timestamp_build_start:         1726485010.215996
    timestamp_build_complete:      1726485019.535536
    timestamp_solve_start:         1726485019.535693
    timestamp_solve_complete:      1726485040.582997

In [9]:
y_label = "Flow in/out (100 GWh)"
df_electricity = (
    (model.results.flow_out.fillna(0) - model.results.flow_in.fillna(0))
    .sel(carriers="electricity")
    .sum("nodes")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame(y_label)
    .reset_index()
)
df_electricity_demand = df_electricity[df_electricity.techs == "demand_elec"]
df_electricity_other = df_electricity[df_electricity.techs != "demand_elec"]

print(df_electricity.head())

     techs           timesteps  Flow in/out (100 GWh)
0  battery 2016-01-01 00:00:00              -0.307658
1  battery 2016-01-01 01:00:00              -0.192895
2  battery 2016-01-01 02:00:00              -0.092589
3  battery 2016-01-01 03:00:00              -0.114076
4  battery 2016-01-01 04:00:00              -0.182358


In [10]:
fig1 = px.bar(
    df_electricity_other,
    x="timesteps",
    y=y_label,
    color="techs",
)
fig1.add_scatter(
    x=df_electricity_demand.timesteps,
    y=-1 * df_electricity_demand[y_label],
    marker_color="black",
    name="demand",
)